# Installing AutoGen

AutoGen supports Python version >= 3.8 and < 3.13. In the below cell we are installing `pyautogen` through pip.

In [1]:
!pip install pyautogen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


# Setting Up The Environment

Here we're using the configuration of GPT-3.5 Turbo model from Azure OpenAI and storing the News API key in environment variable.

GPT Configuration is taken from the Google Colab Secrets.

> **NOTE:** We want to prevent the sensitive News API key from directly being exposed to the LLM model. However, we want code generated by LLM to still have access to the News API Key. Thus, environmental variables are best suited.

In [2]:
from google.colab import userdata
import os

config_list = [
    {
        "model": "gpt-35-turbo-16k",
        "base_url": userdata.get("AZURE_OPENAI_ENDPOINT"),
        "api_key": userdata.get("AZURE_OPENAI_API_KEY"),
        "api_type": "azure",
        "api_version": "2024-02-15-preview"
    }
]

os.environ["NEWS_API_KEY"] = "<NEWSORG_API_KEY>"

# Creating AI Agents

This is where our agents gets defined. Here we are creating two agents:

1. **AssistantAgent:** It acts as an AI model. It can use LLM to generate responses. It can even write code, check the results and suggest fixes.
2. **UserProxyAgent:** It can mimic the user's interaction. Here, we have provided the UserProxyAgent the capability to perform code execution. With options like "code_execution_config" and "llm_config", you can regulate code execution and LLM utilization for response generation.

In [7]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor

assistant = AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list,
        "temperature": 0
    },
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={ "executor": LocalCommandLineCodeExecutor(work_dir="coding") },
    default_auto_reply="Reply 'TERMINATE' if the task is done."
)

# Initiating The Conversation

This is where we start the interaction between the agents.

In [8]:
chat_res = user_proxy.initiate_chat(
    assistant,
    message="""
        Do you know about newsapi.org?
        Can you use that to retrieve the top 10 tech news headlines and the content from techcrunch.com?
        Your API KEY for newsapi.org is stored in os.environ["NEWS_API_KEY"].
    """,
    summary_method="reflection_with_llm"
)

user_proxy (to assistant):


        Do you know about newsapi.org?
        Can you use that to retrieve the top 10 tech news headlines and the content from techcrunch.com?
        Your API KEY for newsapi.org is stored in os.environ["NEWS_API_KEY"].
    

--------------------------------------------------------------------------------
assistant (to user_proxy):

Yes, I am familiar with newsapi.org. To retrieve the top 10 tech news headlines and content from techcrunch.com using the newsapi.org API, you can use the following Python code:

```python
import requests
import os

# Get the API key from the environment variable
api_key = os.environ["NEWS_API_KEY"]

# Define the API endpoint and parameters
url = "https://newsapi.org/v2/top-headlines"
params = {
    "apiKey": api_key,
    "sources": "techcrunch",
    "pageSize": 10
}

# Send the request to the API
response = requests.get(url, params=params)
data = response.json()

# Print the headlines and content
for article in data["articles

# Voila!

Congratualtions on creating your first AutoGen agent to retrieve news from techcrunch.com.